In [2]:
import numpy as np
import pandas as pd 
from datasets import load_dataset
import tqdm as tqdm
import matplotlib.pyplot as plt
import plotly.express as px


c:\Users\franz\anaconda3\envs\ada\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
regression_dataset = load_dataset("ada-datadruids/regression_dataset_tmdb")
regression_dataset_df = regression_dataset['train'].to_pandas()

In [4]:
regression_dataset_df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords', 'based_on_book'],
      dtype='object')

In [5]:
books_movie_dataset = pd.read_csv("../data/final_dataset.csv")
books_movie_dataset.columns

Index(['Unnamed: 0', 'movie_id', 'movie_cluster', 'movie_name', 'release_date',
       'runtime_x', 'language', 'country', 'genres', 'movie_year', 'index',
       'id', 'BookTitle', 'BookYear', 'Author', 'full name', 'tmdb_id',
       'id_goodreads', 'imdbid', 'runtime_y', 'revenue', 'budget', 'rb_ratio',
       'book_id', 'normalized_rating_x', 'standardized_rating_x',
       'normalized_rating_y', 'standardized_rating_y', 'length',
       'review_count'],
      dtype='object')

In [6]:
from utils.general_utils import adjust_for_inflation_final_dataset
books_movie_dataset_cleaned = books_movie_dataset[books_movie_dataset['revenue'] != 0]
books_movie_dataset_cleaned = adjust_for_inflation_final_dataset(books_movie_dataset, books_movie_dataset_cleaned)

In [7]:
book_adaptations = books_movie_dataset_cleaned.groupby(['book_id', 'BookTitle']).size().reset_index(name='adaptation_count')

# Sort by 'adaptation_count' in descending order to see the most frequently adapted books
book_adaptations = book_adaptations.sort_values(by='adaptation_count', ascending=False)

# Display the top books with the most adaptations
print(book_adaptations[book_adaptations.adaptation_count > 1])

        book_id                                   BookTitle  adaptation_count
297    414895.0                       La Planète des singes                 7
629  35031085.0                                Frankenstein                 7
405    897831.0                                The Exorcist                 5
114     34268.0  Peter Pan, or The Boy Who Wouldn't Grow Up                 5
47       7190.0                     Les Trois Mousquetaires                 5
..          ...                                         ...               ...
50       7604.0                                      Lolita                 2
16       2493.0                            The Time Machine                 2
618  19161905.0                                 The Getaway                 2
632  40940649.0                                 I Am Legend                 2
574   6356906.0                     The Apple Dumpling Gang                 2

[87 rows x 3 columns]


In [8]:
Planete_singes=books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='La Planète des singes']
frankenstein = books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='Frankenstein']
exorcist= books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='The Exorcist']
peter_pan=books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=="Peter Pan, or The Boy Who Wouldn't Grow Up"]
trois_mousquetaires=books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookTitle']=='Les Trois Mousquetaires']

In [9]:
books_with_multiple_films = ['La Planète des singes', 'Frankenstein', 'The Exorcist',"Peter Pan, or The Boy Who Wouldn't Grow Up", 'Les Trois Mousquetaires']
books_movie_dataset_cleaned['BookIdentifier'] = books_movie_dataset_cleaned['BookTitle'].where(books_movie_dataset_cleaned['BookTitle'].isin(books_with_multiple_films))
books_filtered = books_movie_dataset_cleaned[books_movie_dataset_cleaned['BookIdentifier'].notnull()]

In [10]:
fig = px.scatter(
    books_filtered,
    x='movie_year',
    y='revenue',
    color='BookIdentifier',
    hover_data={'BookTitle': True, 'BookYear': True, 'movie_year': True},
    title='Revenue for different Books adapted multiple Times',
    labels={'BookYear': 'Book Release Year', 'years_to_movie': 'Years to Movie Release'},
    template='plotly_white'
)

fig.update_traces(
    hovertemplate='<b>Book Title:</b> %{customdata[0]}<br>' +
                  '<b>Book Release Year:</b> %{customdata[1]}<br>' +
                  '<b>Movie Release Year:</b> %{x}<br>' +
                  '<b>Years to Movie Release:</b> %{y}<br>'
)

fig.show()


In [11]:
Planete_singes

,Unnamed: 0,movie_id,movie_cluster,movie_name,release_date,runtime_x,language,country,genres,movie_year,...,book_id,normalized_rating_x,standardized_rating_x,normalized_rating_y,standardized_rating_y,length,review_count,Year,CPI,AdjustedRevenue
142,365,657159,/m/0308n2,Conquest of the Planet of the Apes,1972-06-29,88.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01q03"": ""Cult"", ""/m/06n90"": ""Science Fict...",1972,...,414895.0,0.563380,-0.418388,0.643333,0.293655,268.0,1245.0,1972,41.816667,5.403787e+07
143,366,18618306,/m/0k2r1,Planet of the Apes,1968-02-08,107.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01q03"": ""Cult"", ""/m/01jfsb"": ""Thriller"", ...",1968,...,414895.0,0.830986,1.460964,0.643333,0.293655,268.0,1245.0,1968,34.783333,2.182650e+08
266,654,27279709,/m/0btyf5z,Rise of the Planet of the Apes,2011-08-03,105.0,"{""/m/02h40lc"": ""English Language"", ""/m/0my5"": ...","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2011,...,414895.0,0.774648,1.065311,0.643333,0.293655,268.0,1245.0,2011,224.939167,5.000717e+08
282,692,303928,/m/01s7vd,Escape from the Planet of the Apes,1971-05-21,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/03k9fj"": ""...",1971,...,414895.0,0.591549,-0.220562,0.643333,0.293655,268.0,1245.0,1971,40.491667,7.104585e+07
367,899,709860,/m/034rv2,Battle for the Planet of the Apes,1973-06-15,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/03k9fj"": ""...",1973,...,414895.0,0.478873,-1.011868,0.643333,0.293655,268.0,1245.0,1973,44.400000,4.640565e+07
685,1635,20181491,/m/047wh1,Planet of the Apes,2001-07-26,119.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/07s2s"": ""Time tr...",2001,...,414895.0,0.507042,-0.814041,0.643333,0.293655,268.0,1245.0,2001,177.066667,4.765425e+08
766,1825,152058,/m/013szf,Beneath the Planet of the Apes,1970-05-26,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/02kdv5l"": ...",1970,...,414895.0,0.563380,-0.418388,0.643333,0.293655,268.0,1245.0,1970,38.825000,1.140018e+08


In [13]:
frankenstein.columns

Index(['Unnamed: 0', 'movie_id', 'movie_cluster', 'movie_name', 'release_date',
       'runtime_x', 'language', 'country', 'genres', 'movie_year', 'index',
       'id', 'BookTitle', 'BookYear', 'Author', 'full name', 'tmdb_id',
       'id_goodreads', 'imdbid', 'runtime_y', 'revenue', 'budget', 'rb_ratio',
       'book_id', 'normalized_rating_x', 'standardized_rating_x',
       'normalized_rating_y', 'standardized_rating_y', 'length',
       'review_count', 'Year', 'CPI', 'AdjustedRevenue'],
      dtype='object')